In [1]:
""" Process flood risk data and store on BigQuery. 
-------------------------------------------------------------------------------

Author: Rutger Hofste
Date: 20181204
Kernel: python35
Docker: rutgerhofste/gisdocker:ubuntu16.04

"""

SCRIPT_NAME = "Y2018M12D04_RH_UCW_BQ_V01"
OUTPUT_VERSION = 4

S3_INPUT_PATH = "s3://wri-projects/Aqueduct30/finalData/Wastewater"
INPUT_FILE_NAME = "wastewater_results.csv"

BQ_PROJECT_ID = "aqueduct30"
BQ_OUTPUT_DATASET_NAME = "aqueduct30v01"
BQ_OUTPUT_TABLE_NAME = "{}_v{:02.0f}".format(SCRIPT_NAME,OUTPUT_VERSION).lower()

ec2_input_path = "/volumes/data/{}/input_V{:02.0f}".format(SCRIPT_NAME,OUTPUT_VERSION) 
ec2_output_path = "/volumes/data/{}/output_V{:02.0f}".format(SCRIPT_NAME,OUTPUT_VERSION) 

print("S3_INPUT_PATH: ",S3_INPUT_PATH,
      "\nec2_input_path: ",ec2_input_path,
      "\nec2_output_path: ",ec2_output_path,
      "\nBQ_OUTPUT_DATASET_NAME: ", BQ_OUTPUT_DATASET_NAME,
      "\nBQ_OUTPUT_TABLE_NAME: ",BQ_OUTPUT_TABLE_NAME
      )

S3_INPUT_PATH:  s3://wri-projects/Aqueduct30/finalData/Wastewater 
ec2_input_path:  /volumes/data/Y2018M12D04_RH_UCW_BQ_V01/input_V04 
ec2_output_path:  /volumes/data/Y2018M12D04_RH_UCW_BQ_V01/output_V04 
BQ_OUTPUT_DATASET_NAME:  aqueduct30v01 
BQ_OUTPUT_TABLE_NAME:  y2018m12d04_rh_ucw_bq_v01_v04


In [2]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2019M04D05 UTC 10:15


'3.5.4 |Anaconda, Inc.| (default, Nov 20 2017, 18:44:38) \n[GCC 7.2.0]'

In [3]:
!rm -r {ec2_input_path}
!rm -r {ec2_output_path}
!mkdir -p {ec2_input_path}
!mkdir -p {ec2_output_path}

rm: cannot remove '/volumes/data/Y2018M12D04_RH_UCW_BQ_V01/input_V04': No such file or directory
rm: cannot remove '/volumes/data/Y2018M12D04_RH_UCW_BQ_V01/output_V04': No such file or directory


In [4]:
!aws s3 cp {S3_INPUT_PATH} {ec2_input_path} --recursive 

download: s3://wri-projects/Aqueduct30/finalData/Wastewater/wastewater_results.CPG to ../../../../data/Y2018M12D04_RH_UCW_BQ_V01/input_V04/wastewater_results.CPG
download: s3://wri-projects/Aqueduct30/finalData/Wastewater/wastewater_results.sbn to ../../../../data/Y2018M12D04_RH_UCW_BQ_V01/input_V04/wastewater_results.sbn
download: s3://wri-projects/Aqueduct30/finalData/Wastewater/README.txt to ../../../../data/Y2018M12D04_RH_UCW_BQ_V01/input_V04/README.txt
download: s3://wri-projects/Aqueduct30/finalData/Wastewater/wastewater_results.shp.xml to ../../../../data/Y2018M12D04_RH_UCW_BQ_V01/input_V04/wastewater_results.shp.xml
download: s3://wri-projects/Aqueduct30/finalData/Wastewater/wastewater_results.shx to ../../../../data/Y2018M12D04_RH_UCW_BQ_V01/input_V04/wastewater_results.shx
download: s3://wri-projects/Aqueduct30/finalData/Wastewater/wastewater_results.sbx to ../../../../data/Y2018M12D04_RH_UCW_BQ_V01/input_V04/wastewater_results.sbx
download: s3://wri-projects/Aqueduct30/final

In [5]:
import os
import pandas as pd
import numpy as np
from google.cloud import bigquery

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/.google.json"
os.environ["GOOGLE_CLOUD_PROJECT"] = "aqueduct30"
client = bigquery.Client(project=BQ_PROJECT_ID)

In [6]:
files = os.listdir(ec2_input_path)

In [7]:
input_path = "{}/{}".format(ec2_input_path,INPUT_FILE_NAME)

In [8]:
df = pd.read_csv(input_path)

In [9]:
df.dtypes

ADM0_A3               object
Country               object
Exclude               object
Percent_Connected    float64
Untreated            float64
Primary              float64
Secondary            float64
Tertiary             float64
WW_raw               float64
WW_s                 float64
WW_cat                object
dtype: object

In [10]:
df.head()

,ADM0_A3,Country,Exclude,Percent_Connected,Untreated,Primary,Secondary,Tertiary,WW_raw,WW_s,WW_cat
0,ABW,Aruba,No,0.0,100.0,0.0,0.0,0.0,-1.000000,5.000000,Low to no wastewater collected
1,AFG,Afghanistan,No,1.0,0.0,100.0,0.0,0.0,-1.000000,5.000000,Low to no wastewater collected
2,AGO,Angola,No,1.0,100.0,0.0,0.0,0.0,-1.000000,5.000000,Low to no wastewater collected
3,AIA,Anguilla,No,0.0,100.0,0.0,0.0,0.0,-1.000000,5.000000,Low to no wastewater collected
4,ALB,Albania,No,66.7,34.9,32.6,32.6,0.0,0.565783,1.885943,Low to medium (0.3 to 60%)


In [11]:
# WW -> UCW


# raw -> raw.
# s -> score.
# None -> cat.
# cat -> label. 

In [12]:
df_out = df.rename(columns={"ADM0_A3":"adm0_a3",
                            "WW_raw":"ucw_raw",
                            "WW_s":"ucw_score",
                            "WW_cat":"ucw_label"})

In [13]:
df_out.drop(columns=["Exclude",
                     "Percent_Connected",
                     "Untreated",
                     "Primary",
                     "Secondary",
                     "Tertiary"],inplace=True)

In [14]:
def update_labels(label):
    # Update label to correct format.
    if label == "Low to no wastewater collected":
        new_label = "No to Low Wastewater Collected"
    elif label == "Low (< 30%)":
        new_label = "Low (<30%)"
    elif label == "Low to medium (0.3 to 60%)":
        new_label = "Low - Medium (30-60%)"
    elif label == "Medium to high (0.6 to 90%)":
        new_label = "Medium - High (60-90%)"
    elif  label == "High (0.9 to 100%)":
        new_label = "High (90-100%)"
    elif label == "Extremely High (100%)":
        new_label =  "Extremely High (100%)"
    else:
        new_label = "error, check script"
    return new_label

def category_from_label(label):
    # get cat from label
    if label == "Low to no wastewater collected":
        cat = -1
    elif label == "Low (< 30%)":
        cat = 0
    elif label == "Low to medium (0.3 to 60%)":
        cat = 1
    elif label == "Medium to high (0.6 to 90%)":
        cat = 2
    elif  label == "High (0.9 to 100%)":
        cat = 3
    elif label == "Extremely High (100%)":
        cat = 4
    else:
        cat = -9999
    return cat
    



In [15]:
df_out["ucw_cat"] = df_out["ucw_label"].apply(category_from_label)


In [16]:
df_out["ucw_label"] = df_out["ucw_label"].apply(update_labels)

In [17]:
df_out = df_out.reindex(sorted(df_out.columns), axis=1)

In [18]:
df_out.head()

,Country,adm0_a3,ucw_cat,ucw_label,ucw_raw,ucw_score
0,Aruba,ABW,-1,No to Low Wastewater Collected,-1.000000,5.000000
1,Afghanistan,AFG,-1,No to Low Wastewater Collected,-1.000000,5.000000
2,Angola,AGO,-1,No to Low Wastewater Collected,-1.000000,5.000000
3,Anguilla,AIA,-1,No to Low Wastewater Collected,-1.000000,5.000000
4,Albania,ALB,1,Low - Medium (30-60%),0.565783,1.885943


In [19]:
df_out.loc[(df_out['ucw_raw'] == -1.0) ,'ucw_cat'] = -1


In [20]:
df_out.head()

,Country,adm0_a3,ucw_cat,ucw_label,ucw_raw,ucw_score
0,Aruba,ABW,-1,No to Low Wastewater Collected,-1.000000,5.000000
1,Afghanistan,AFG,-1,No to Low Wastewater Collected,-1.000000,5.000000
2,Angola,AGO,-1,No to Low Wastewater Collected,-1.000000,5.000000
3,Anguilla,AIA,-1,No to Low Wastewater Collected,-1.000000,5.000000
4,Albania,ALB,1,Low - Medium (30-60%),0.565783,1.885943


In [21]:
destination_table = "{}.{}".format(BQ_OUTPUT_DATASET_NAME,BQ_OUTPUT_TABLE_NAME)

In [22]:
destination_table

'aqueduct30v01.y2018m12d04_rh_ucw_bq_v01_v04'

In [23]:
df_out.to_gbq(destination_table=destination_table,
          project_id=BQ_PROJECT_ID,
          chunksize=10000,
          if_exists="replace")

1it [00:06,  6.07s/it]


In [24]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

0:00:16.978664


Previous runs:  
0:00:18.766466